<h3>zeroMQ remote API: Información preliminar</h3>
<br>Entre las múltiples opciones de comunicación con CoppeliaSIm, la zeroMQ remote API entrega funcionalidades que facilitan la interacción con scripts de Python y con otros lenguajes de programación.
<br>Las instrucciones para instalar la librería se encuentra aquí: <a>https://www.coppeliarobotics.com/helpFiles/en/zmqRemoteApiOverview.htm</a>
<br>La descripción detallada de las funciones de la API se encuentra aquí: <a>https://www.coppeliarobotics.com/helpFiles/en/apiFunctions.htm</a>

<h3>1. Instrucciones mínimas para comunicarse con CoppeliaSim</h3>

Varios ejemplos del uso de la zeroMQ remote API pueden consultarse aquí: <a>https://github.com/CoppeliaRobotics/zmqRemoteApi/tree/master/clients/python</a>

In [ ]:
# https://www.coppeliarobotics.com/helpFiles/en/apiFunctions.htm
# https://www.coppeliarobotics.com/helpFiles/en/zmqRemoteApiOverview.htm

In [ ]:
# librerías requeridas
import serial  # librería pyserial, para comunicación con los puertos serial < pip install pyserial >
import time    # para el manejo de eventos de tiempo, nativa de Python
import pandas as pd  # para el manejo de archivos de datos < pip install pandas >
import matplotlib.pyplot as plt # para realizar gráficos < pip install matplotlib >


In [ ]:
# 1. Importamos la librería para comunicaciones remotas
# NOTA 1: Se requiere la instalación previa de pyzmq y cbor.
# NOTA 2: Si la librería no está direccionada en el PATH de python, copiar 
#         la carpeta de librería 'zmqRemoteApi' junto a este cuaderno de Jypyter. 
#         'zmqRemoteApi' para Python se encuentra en:
#                 *ruta de instalacion*\CoppeliaSimEdu\programming\zmqRemoteApi\clients\python\
import zmqRemoteApi

# 2. Se crea un cliente para conectarse al servidor zmqRemoteApi de CoppeliaSim
# los parámetros por defecto son: host='localhost', port=23000
client = zmqRemoteApi.RemoteAPIClient()

# 3. Obtener el objeto remoto 'sim':
sim = client.getObject('sim')

# 4. Llamar a una funcion de la API:
h = sim.getObject('/YuMi')

In [ ]:
# funciones para enviar y recibir mensajes en formato de texto, codificado en UTF-8
# los mensajes utilizan como terminador un caracter de final de línea '\n'
def encode_send(ser, texto):
    enc = f'{texto}\n'.encode('UTF-8')
    ser.write(enc)

def decode_response(ser):
    message_from_serial = ser.readline()
    res = message_from_serial[:-1].decode()
    return res

<h3>2. Usando la API remota para mover nuestro robot
Y Obteniendo una lectura de datos </h3>


In [4]:
# Cargamos la librería, creamos un cliente y obtenemos acceso a sim
import zmqRemoteApi
client = zmqRemoteApi.RemoteAPIClient()
sim = client.getObject('sim')

In [ ]:
# Obtenemos los manejadores para las articulaciones y el actuador final
leftjoint1 = sim.getObject('/YuMi/leftJoint1')  #Cada nombre depende del tipo de robot/articulación que queramos llegar 
leftjoint2 = sim.getObject('/YuMi/leftJoint2')
leftjoint3 = sim.getObject('/YuMi/leftJoint3')

print(leftjoint1, leftjoint2, leftjoint3)

<h3>3. Comprobar datos del análogo </h3>

In [ ]:
# Esquema básico para control de pines I/O en modo request-response

# 1. Inicializa el puerto de comunicación y espera a que esté listo
serialport = serial.Serial('COM4', 9600)
time.sleep(0.1)   # tiempo de espera recomendado: 100 ms

# 2. Preparamos el mensaje a ser transmitido
led_verde = 0  # 0 = apagado, otro valor = encendido
led_rojo = 0    # nivel PWM entre 0 y 255
message_to_serial = str(led_verde) + ',' + str(led_rojo)

# 3. Enviamos al puerto y esperamos la respuesta
encode_send(serialport, message_to_serial)
message_from_serial = decode_response(serialport)
encode_send(serialport, message_to_serial)
message_from_serial = decode_response(serialport)

# 4. Procesamos la respuesta
print(f'enviado: {message_to_serial}')

data = message_from_serial.split(';')
val_x = int(data[0])
val_y = int(data[1])
val_sw = int(data[2])
if val_sw < 1023: 
    val_sw = 0
else: val_sw = 1

print(f'val_x = {val_x}')
print(f'val_y = {val_y}')
print(f'val_sw = {val_sw}')

# 5. Cierra el puerto serial
serialport.close()
print('el puerto se ha cerrado correctamente')

<h3>Práctica con el Robot </h3>


In [66]:
#Movimiento del robot
#se moverán los 3 joints a la vez
sim.setJointTargetPosition(joint1, 90*3.14159/180)
sim.setJointTargetPosition(joint2, 0.15) #debido a que joint2 es prismático su movimiento se mide en metros
sim.setJointTargetPosition(joint3, 50*3.14159/180)

1

In [ ]:
#Búsqueda de posición del robot
xyz_joint1 = sim.getObjectPosition(joint1, sim.handle_world)
xyz_joint2 = sim.getObjectPosition(joint2, sim.handle_world)
xyz_joint3 = sim.getObjectPosition(joint3, sim.handle_world)
xyz_endEffector = sim.getObjectPosition(endEffector, sim.handle_world)

#Imprimir las coordenadas en pantalla
print(f'Joint1: {xyz_joint1}', f'joint2: {xyz_joint2}', f'joint3: {xyz_joint3}', f'endEffector: {xyz_endEffector}')

In [ ]:
#Imprimir las coordenadas en pantalla
print(f'joint1: {xyz_joint1}')
print(f'joint2: {xyz_joint2}')
print(f'joint3: {xyz_joint3}')
print(f'endEffector: {xyz_endEffector}')